# Bond Future Fundamentals

This notebook demonstrates how to access and use the **Bond Future** functionalities within the **LSEG Financial Analytics SDK**.

Bond futures are financial derivatives that obligate the contract holder to purchase or sell a bond on a specified date at a predetermined price. A bond future contract trades on a future exchange market and is bought or sold through a brokerage firm that offers future trading.

**You will be able to:**
- Define Bond Future with basic parameters
- Configure pricing parameters
- Evaluate Bond Future Analytics

## Imports


Import the following necessary modules:

- `lseg_analytics.pricing.instruments.bond_future` - for Bond Future instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import bond_future as bf

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bond Repo instrument you need to follow a structured 4-step process:

1. **Bond Definition** - Specify the underlying Bond parameters 
2. **Future Definition** - Specify the Future parameters
3. **Future Definition Instrument** - Create the instrument object using the Future Definition
3. **Pricing Preferences** - Configure pricing parameters, optional


In [2]:
# 1.Define an underlying Bond instrument
fixed_bond_definition = bf.BondDefinition(
    notional_ccy = "EUR",
    issue_date = dt.datetime.strptime("2025-01-01", "%Y-%m-%d"),
    end_date = dt.datetime.strptime("2030-01-01", "%Y-%m-%d"),
    fixed_rate_percent = 2,
    interest_payment_frequency = bf.InterestPaymentFrequencyEnum.QUARTERLY,
    interest_calculation_method = bf.InterestCalculationMethodEnum.DCB_ACTUAL_ACTUAL

)

underlying_bond = bf.BondFutureUnderlyingContract(
        instrument_definition = fixed_bond_definition,
        instrument_type = "Bond"
)
print("1 - Underlying Bond definition created")

# 2.Define Future instrument
future_definition = bf.BondFutureDefinition(
    instrument_code = "FOATc1",  # Mandatory field
    underlying_instruments = [underlying_bond]
)
print("2 - Future instrument defined")

# 3.Create the Future Instrument from the defintion
future_instrument = bf.BondFutureDefinitionInstrument(definition = future_definition)
print("3 - Future Instrument created")

# 4. Configure pricing parameters
pricing_params = bf.BondFuturePricingParameters(
    valuation_date = dt.datetime.strptime("2025-07-18", "%Y-%m-%d"),                
)
print("4 - Pricing parameters configured")

1 - Underlying Bond definition created
2 - Future instrument defined
3 - Future Instrument created
4 - Pricing parameters configured


## Request Execution

In [3]:
#  Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

# Execute the calculation using the price() function with error handling
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = bf.price(
        definitions=[future_instrument],
        pricing_preferences=pricing_params
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Bond Future pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Bond Future pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instument and Underlying Bond definitions set above. 

 - **pricingPreferences**: Valuation date.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`). If not specified all fields are returned.
   - **description**: Instrument summary and also fields not specified with the defaulted values used in the pricing.
   - **nominalMeasures**: list of Bond future sensitivities
   - **pricingAnalysis**: list of analytics used in the pricing
   - **valuation**: Market valuation details on Bond future
   - **error**: root cause of the issue if any

We give below the details on description, pricing analysis, nominal measures and valuation

In [4]:
# Access the description object
description = response.data.analytics[0].description
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentCode": "FOATc1",
    "instrumentDescription": "FOAT DEC5",
    "endDate": "2025-12-08T00:00:00Z",
    "deliveryStartDate": "2025-12-10",
    "deliveryEndDate": "2025-12-10",
    "notionalCcy": "EUR",
    "notionalAmount": 1000000.0,
    "contractSize": 100000.0,
    "couponRatePercent": 6.0
}


In [5]:
# Access the pricing analysis object
pricing_analysis = response.data.analytics[0]["pricingAnalysis"]
df_pricing_analysis = pd.DataFrame(list(pricing_analysis.items()), columns=["Fields", "Value"])
display(df_pricing_analysis.head(9))

,Fields,Value
0,marketDataDate,2025-07-18T00:00:00Z
1,valuationDate,2025-07-18T00:00:00Z
2,cheapestToDeliver,Bond#1
3,conversionFactor,0.856255
4,price,122.84
5,fairPrice,121.812169
6,invoicePrice,105.182364
7,yieldPercent,3.284616
8,impliedYieldPercent,0.688903


In [6]:
# Access the nominal measures object
nominal_measures = response.data.analytics[0]["nominalMeasures"]
df_nominal_measures = pd.DataFrame(list(nominal_measures.items()), columns=["Fields", "Value"])
display(df_nominal_measures)

,Fields,Value
0,convexity,76.553562
1,dv01Bp,9.569231
2,duration,8.045876
3,modifiedDuration,7.789996


In [7]:
valuation = response.data.analytics[0].valuation
df_valuation = pd.DataFrame(list(valuation.items()), columns=["Fields", "Value"])
display(df_valuation)

,Fields,Value
0,marketValueInDealCcy,1228400.0
1,marketValueInReportCcy,1228400.0


## Appendix: Main Functions Reference

The following is a list of main classes, enums, and functions available after importing from `lseg_analytics.pricing.instruments.bond_future`:

**Core Classes:**
- `BondFutureDefinition` - Define bond future instrument specifications
- `BondDefinition` - Define underlying bond instrument specifications
- `BondFutureUnderlyingContract` - Define the bond future underlying instrument
- `BondFutureDefinitionInstrument` - Create bond future instrument from RepoDefinition
- `BondFuturePricingParameters` - Configure pricing parameters and market data settings 
- `BondPricingParameters` - Configure pricing parameters and market data settings for the underlying bond

**Response Classes:**
- `BondFutureCalculationResponse` - Main response container
- `BondFutureAnalyticsResponseData` - Detailed analytics and calculations
- `BondFutureAnalyticsResponseWithError` - Analytics response with error
- `BondFutureDescription` - Instrument summary
- `BondFuturePricingAnalysis` - Analytics used in the pricing
- `BondFutureValuation` - Individual instrument valuation results
- `BondFutureNominalMeasures` - Bond future risk metrics
- `BondFuturePricingParameters` - Pricing parameters


**Main Functions:**
- `price()` - Execute Bond future pricing calculation